In [1]:

import json, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

belief_path = r"C:\Users\kuzey\OneDrive\Masaüstü\Curley's Wife\data\belief.txt"
jsonl_path = r"C:\Users\kuzey\OneDrive\Masaüstü\Curley's Wife\data\mistral_curleys_dataset.jsonl"
output_dir = r"C:\Users\kuzey\OneDrive\Masaüstü\Curley's Wife\curleyswife_gpt2medium"

with open(belief_path, "r", encoding="utf-8") as f:
    beliefs = f.read().strip()

with open(jsonl_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]

for entry in raw_data:
    entry["text"] = beliefs + "\n\n" + entry["prompt"] + entry["completion"]

dataset = Dataset.from_list([{"text": e["text"]} for e in raw_data])

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.to("cuda")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    fp16=True,
    save_strategy="epoch",
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\kuzey\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kuzey\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Step,Training Loss
10,2.309900
20,0.400000
30,0.015800
40,0.001800
50,0.001000
60,0.000800
70,0.001000
80,0.000500
90,0.000300


("C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\tokenizer_config.json",
 "C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\special_tokens_map.json",
 "C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\vocab.json",
 "C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\merges.txt",
 "C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\added_tokens.json",
 "C:\\Users\\kuzey\\OneDrive\\Masaüstü\\Curley's Wife\\curleyswife_gpt2medium\\tokenizer.json")

In [ ]:
#test

In [14]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_chat_model_cpu(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(model_path).to("cpu")
    model.eval()
    return tokenizer, model

with open(r"C:\Users\kuzey\OneDrive\Masaüstü\CurleysWife\data\belief.txt", "r", encoding="utf-8") as f:
    beliefs = f.read().strip()

chat_history = []

def safe_chat_cpu(user_input, tokenizer, model, max_tokens=100):
    global chat_history

    prompt = "[Memory]\n" + beliefs + "\n\n[Chat History]\n"
    for turn in chat_history:
        prompt += f"User: {turn['user']}\nCurley's Wife: {turn['bot']}\n"
    prompt += f"User: {user_input}\nCurley's Wife:"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024 - max_tokens)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.75,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    reply = decoded.split("Curley's Wife:")[-1].strip().split("\n")[0]

    chat_history.append({"user": user_input, "bot": reply})
    return reply


In [15]:
chat_history = []


In [16]:
tokenizer, model = load_chat_model_cpu(r"C:\Users\kuzey\OneDrive\Masaüstü\CurleysWife\model\curleyswife_gpt2medium")
print(safe_chat_cpu("Do you support Trump or Joe Biden?", tokenizer, model))

I’m not sure.


In [14]:
chat_history

[{'user': 'Who is your favorite person?',
  'bot': "I like to think I'm respectable."}]